Gratidão aos colegas Thiago Soares Laitz e Hugo (hugo@maritaca.ai) pelo apoio e código base fornecidos

# Install

In [ ]:
#!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00


# Imports

In [1]:
from tqdm import tqdm

# Organizando o ambiente

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun 27 00:29:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   49C    P8    26W / 370W |     60MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory


In [4]:
def mostra_memoria(lista_mem=['cpu','gpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'.
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])

  Autor: Marcus Vinícius Borela de Castro

  """
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [5]:
mostra_memoria()

Your runtime RAM in gb: 
 total 67.35
 available 55.4
 used 10.82
 free 35.72
 cached 19.93
 buffers 0.88
/nGPU
Tue Jun 27 00:29:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   50C    P8    37W / 370W |     60MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+

### Vinculando pasta do google drive para salvar dados

In [6]:
import os

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /home/borela/fontes/ind-ir/code/train


## Fixando as seeds

In [9]:
import random
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader

In [10]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU.

  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.

  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)

  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [11]:
num_semente=123
inicializa_seed(num_semente)

## Definindo Hiperparâmetros iniciais

In [12]:
def inicia_hparam()->dict:
  # Inicialização dos parâmetros
  hparam = {}
  hparam["num_workers_dataloader"] = 0
  hparam["device"] = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  if torch.cuda.is_available(): print(torch. cuda. get_device_name(hparam["device"]))
  return hparam

In [13]:
hparam=inicia_hparam()

NVIDIA GeForce RTX 3090


## Preparando para debug e display

https://zohaib.me/debugging-in-google-collab-notebook/

In [14]:
# !pip install -Uqq ipdb
# import ipdb

In [15]:
import pandas as pd

In [ ]:
# !pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00


In [16]:
import transformers

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [18]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True)
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345,
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite
    uma melhor compreensão dos números envolvidos.
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica,
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada,
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada.
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona,
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  transformers.utils.logging.set_verbosity_info()


  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a
    necessidade de cada tarefa.

    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning

    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  %xmode Verbose

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain,
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.

      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha,
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código),
        step (para entrar em uma função chamada na próxima linha de código)
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [19]:
config_display()

In [20]:
config_debug()

Exception reporting mode: Verbose


In [ ]:
os.environ['NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE'] = 'TRUE'
os.environ['NEPTUNE_PROJECT'] = 'marcusborela/IA386DD'
os.environ['NEPTUNE_API_TOKEN'] = getpass.getpass('Informe NEPTUNE_API_TOKEN')

tag_contexto_rastro = 'Aula9_InPars'
neptune_version = 0
### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)


def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário 
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]
if neptune_version == 0:
  import neptune.new as neptune  
  class NeptuneRastroRun():
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].log(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          """
          se dataloader devolver x e y:
          """
          x_, y_ = next(iter(loader_train))
          x_ = x_.to(self.device)
          outputs = model(x_)
          """
          # se dataloader devolver dict:
          dados_ = next(iter(loader_train))
          outputs = model(dados_['x'].to(self.device))
          #outputs = model(x_['input_ids'].to(self.device), x_['attention_mask'].to(self.device))
          """
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      



      def stop(self):
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()

if neptune_version == 1:
  import neptune
  class NeptuneRastroRun():
      """
        Classe para geração de rastro de experimento utilizando a ferramenta Neptune.

        Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664),
        autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)

        Attributes:
        -----------
        se_geracao_rastro : bool
            Indica se deve ser gerado rastro de experimento. 
        neptune_project : str
            Nome do projeto criado no Neptune. 
        tag_contexto_rastro : str
            Nome da tag utilizada para identificar o experimento.
        neptune_api_token : str
            Token utilizado para autenticação na API do Neptune. 
        run_neptune : object
            Objeto que representa o experimento no Neptune.
        device : str
            Dispositivo utilizado para o treinamento do modelo.
        tmpDir : str
          Diretório temporário utilizado para salvar gráfico do modelo.          
      """
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        """
          Método construtor da classe NeptuneRastroRun.
          
          Args:
          - parm_params: dicionário contendo os parâmetros do modelo.
          - parm_lista_tag: lista contendo tags adicionais para o experimento.
        """      
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          # self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        """
        Retorna a instância do objeto run_neptune.
        """      
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        """
        Ativa a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        """
        Define o contexto para a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        """
        Desativa a geração de rastro.
        """      
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        """
          Retorna o status da geração de rastro.
          
          Returns:
          - True se a geração de rastro está ativada, False caso contrário.
        """      
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        """
          Retorna a tag do contexto de rastro.
        """      
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        """
        Inicia o contexto de execução no Neptune.

        Args:
            neptune_project (str): Nome do projeto no Neptune.
            tag_contexto_rastro (str): Tag que identifica o contexto de execução no Neptune.
            neptune_api_token (str): Token de acesso à API do Neptune.

        Raises:
            AssertionError: Caso a tag_contexto_rastro possua um ponto (.), 
              o que pode gerar erros na gravação de arquivo.
        """      
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        """
          Salva as métricas no Neptune Run caso a geração de rastro esteja ativa.

          Parameters
          ----------
          parm_metricas: dict
              Dicionário contendo as métricas a serem salvas. As chaves devem ser os nomes das métricas e os valores devem ser
              os valores das métricas.
        """
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].append(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        """
          Gera um gráfico do modelo e o envia para o Neptune. 
          Para gerar o gráfico, um forward pass é realizado em um batch de exemplos 
          de treino e o resultado é renderizado como um gráfico de nós conectados. 
          O gráfico é salvo em um arquivo .png e enviado para o Neptune como um arquivo anexo.

          Args:
              loader_train (torch.utils.data.DataLoader): DataLoader do conjunto de treinamento.
              model (torch.nn.Module): Modelo a ser visualizado.
          
          Pendente:
            Evolui para usar from io import StringIO (buffer = io.StringIO()) ao invés de tempdir 
        """    
        return

        """
        falta ajustar make_dot
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          batch = next(iter(loader_train))
          # falta generalizar linha abaixo. Criar função que recebe modelo e batch como parâmetro?
          outputs = model(input_ids=batch['input_ids'].to(hparam['device']), attention_mask=batch['attention_mask'].to(hparam['device']), token_type_ids=batch['token_type_ids'].to(hparam['device']), labels=batch['labels'].to(hparam['device']))
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      
        """


      def stop(self):
        """
          Para a execução do objeto Neptune. Todos os experimentos do Neptune são sincronizados com o servidor, e nenhum outro 
          experimento poderá ser adicionado a este objeto após a chamada a este método.
        """
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()

### Definindo parâmetros para o rastro


NeptuneRastroRun.inicia_contexto(os.environ['NEPTUNE_PROJECT'], tag_contexto_rastro,  os.environ['NEPTUNE_API_TOKEN'])
#NeptuneRastroRun.desativa_geracao_rastro()

device(type='cuda', index=0)

## Rastro (neptune.ai)

Gerado rastro da execução no Neptune (detalhes no artigo [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664))


### Importação de libraries para Rastro

In [ ]:
# !pip install neptune-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.4/450.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.1 MB/s eta 0:00:00


## Rastro (neptune.ai)

Gerado rastro da execução no Neptune (detalhes no artigo [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664))

### Importação de libraries para Rastro


In [115]:
import getpass, copy, tempfile, re
os.environ['NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE'] = 'TRUE'
os.environ['NEPTUNE_PROJECT'] = 'marcusborela/IA386DD'
os.environ['NEPTUNE_API_TOKEN'] = getpass.getpass('Informe NEPTUNE_API_TOKEN')

tag_contexto_rastro = 'Aula9_InPars'
neptune_version = 0


In [ ]:
### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)




In [118]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário 
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]
if neptune_version == 0:
  import neptune.new as neptune  
  class NeptuneRastroRun():
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].log(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          """
          se dataloader devolver x e y:
          """
          x_, y_ = next(iter(loader_train))
          x_ = x_.to(self.device)
          outputs = model(x_)
          """
          # se dataloader devolver dict:
          dados_ = next(iter(loader_train))
          outputs = model(dados_['x'].to(self.device))
          #outputs = model(x_['input_ids'].to(self.device), x_['attention_mask'].to(self.device))
          """
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      



      def stop(self):
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()

if neptune_version == 1:
  import neptune
  class NeptuneRastroRun():
      """
        Classe para geração de rastro de experimento utilizando a ferramenta Neptune.

        Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664),
        autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)

        Attributes:
        -----------
        se_geracao_rastro : bool
            Indica se deve ser gerado rastro de experimento. 
        neptune_project : str
            Nome do projeto criado no Neptune. 
        tag_contexto_rastro : str
            Nome da tag utilizada para identificar o experimento.
        neptune_api_token : str
            Token utilizado para autenticação na API do Neptune. 
        run_neptune : object
            Objeto que representa o experimento no Neptune.
        device : str
            Dispositivo utilizado para o treinamento do modelo.
        tmpDir : str
          Diretório temporário utilizado para salvar gráfico do modelo.          
      """
      se_geracao_rastro = True 
      neptune_project = ""
      tag_contexto_rastro = ""
      neptune_api_token = ""

      def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
        """
          Método construtor da classe NeptuneRastroRun.
          
          Args:
          - parm_params: dicionário contendo os parâmetros do modelo.
          - parm_lista_tag: lista contendo tags adicionais para o experimento.
        """      
        # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
        if self.__class__.se_geracao_rastro:      
          self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
          self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
          vparams = copy.deepcopy(parm_params)
          if "optimizer" in vparams:
            vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
          if 'criterion'  in vparams:
            vparams["criterion"] = str(vparams["criterion"])
          if 'scheduler'  in vparams:
            vparams["scheduler"] = str(type(vparams["scheduler"]))
          if 'device' in vparams:
            vparams['device'] = str(vparams["device"])
          self.device = vparams["device"]
          for tag in parm_lista_tag:
            self.run_neptune['sys/tags'].add(tag)
          self.run_neptune['parameters'] = vparams
          # self.tmpDir = tempfile.mkdtemp()

      @property
      def run():
        """
        Retorna a instância do objeto run_neptune.
        """      
        return self.run_neptune

      @classmethod
      def ativa_geracao_rastro(cls):
        """
        Ativa a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def def_contexto(cls):
        """
        Define o contexto para a geração de rastro.
        """      
        cls.se_geracao_rastro = True      

      @classmethod
      def desativa_geracao_rastro(cls):
        """
        Desativa a geração de rastro.
        """      
        cls.se_geracao_rastro = False      

      @classmethod
      def retorna_status_geracao_rastro(cls):
        """
          Retorna o status da geração de rastro.
          
          Returns:
          - True se a geração de rastro está ativada, False caso contrário.
        """      
        return cls.se_geracao_rastro      

      @classmethod
      def retorna_tag_contexto_rastro(cls):
        """
          Retorna a tag do contexto de rastro.
        """      
        return cls.tag_contexto_rastro 

      @classmethod
      def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
        """
        Inicia o contexto de execução no Neptune.

        Args:
            neptune_project (str): Nome do projeto no Neptune.
            tag_contexto_rastro (str): Tag que identifica o contexto de execução no Neptune.
            neptune_api_token (str): Token de acesso à API do Neptune.

        Raises:
            AssertionError: Caso a tag_contexto_rastro possua um ponto (.), 
              o que pode gerar erros na gravação de arquivo.
        """      
        assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
        cls.neptune_api_token = neptune_api_token
        cls.tag_contexto_rastro = tag_contexto_rastro
        cls.neptune_project = neptune_project

      def salva_metrica(self, parm_metricas={}):
        """
          Salva as métricas no Neptune Run caso a geração de rastro esteja ativa.

          Parameters
          ----------
          parm_metricas: dict
              Dicionário contendo as métricas a serem salvas. As chaves devem ser os nomes das métricas e os valores devem ser
              os valores das métricas.
        """
        #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
        if self.__class__.se_geracao_rastro:
          for metrica, valor in parm_metricas.items(): 
            self.run_neptune[metrica].append(valor)
  
      def gera_grafico_modelo(self, loader_train, model):
        """
          Gera um gráfico do modelo e o envia para o Neptune. 
          Para gerar o gráfico, um forward pass é realizado em um batch de exemplos 
          de treino e o resultado é renderizado como um gráfico de nós conectados. 
          O gráfico é salvo em um arquivo .png e enviado para o Neptune como um arquivo anexo.

          Args:
              loader_train (torch.utils.data.DataLoader): DataLoader do conjunto de treinamento.
              model (torch.nn.Module): Modelo a ser visualizado.
          
          Pendente:
            Evolui para usar from io import StringIO (buffer = io.StringIO()) ao invés de tempdir 
        """    
        return

        """
        falta ajustar make_dot
        if self.__class__.se_geracao_rastro: 
          # efetuar um forward 
          batch = next(iter(loader_train))
          # falta generalizar linha abaixo. Criar função que recebe modelo e batch como parâmetro?
          outputs = model(input_ids=batch['input_ids'].to(hparam['device']), attention_mask=batch['attention_mask'].to(hparam['device']), token_type_ids=batch['token_type_ids'].to(hparam['device']), labels=batch['labels'].to(hparam['device']))
          nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
          make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
          self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
          self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      
        """


      def stop(self):
        """
          Para a execução do objeto Neptune. Todos os experimentos do Neptune são sincronizados com o servidor, e nenhum outro 
          experimento poderá ser adicionado a este objeto após a chamada a este método.
        """
        if self.__class__.se_geracao_rastro:         
          self.run_neptune.stop()



In [119]:
### Definindo parâmetros para o rastro
NeptuneRastroRun.inicia_contexto(os.environ['NEPTUNE_PROJECT'], tag_contexto_rastro,  os.environ['NEPTUNE_API_TOKEN'])
#NeptuneRastroRun.desativa_geracao_rastro()

### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)



In [120]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]

In [121]:
def gera_tag_rastro_experiencia_treino(parm_aula: str, hparam: dict) -> str:
    """
    Gera uma string formatada com informações de hiperparâmetros para ser usada como tag de rastro de experiência de treino.

    Args:
        parm_aula (str): Nome da aula que está sendo treinada.
        hparam (dict): Dicionário contendo os hiperparâmetros utilizados no treinamento.

    Returns:
        str: String formatada com as informações de hiperparâmetros.

    Uso:

    hparam['lista_tag_rastro_experiencia_treino'] =        gera_tag_rastro_experiencia_treino(parm_aula='aula7', hparam=hparam)
    """
    # Inicializa uma lista vazia para armazenar as tags
    lista_tag = []

    # Lista com as chaves dos hiperparâmetros que serão utilizados
    lista_chaves = ['embed_dim', 'leiaute_input', 'dim_feedforward', 'max_seq_length', 'ind_activation_function', 'batch_size', 'learning_rate', 'weight_decay', 'amsgrad', 'decrease_factor_lr', 'max_examples', 'eval_every_steps']

    # Itera pelas chaves da lista e cria uma string com a chave e o valor correspondente em hparam,
    # adicionando essa string à lista_tag
    for chave in lista_chaves:
        if chave in hparam:
          tag = f"{chave} {hparam[chave]}"
          lista_tag.append(tag)

    # Concatena a lista de tags em uma única string, separando cada tag por '|',
    # e adicionando o nome da aula como prefixo
    tag_formatada = f"{parm_aula}|" + "|".join(lista_tag)

    return tag_formatada




# Data load

In [34]:
PATH_LOCAL_DATA = '../..'

In [37]:
# path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data/train_data_juris_tcu_index_bm25.csv'

# PATH_TRAIN_DATA_ZIP = f"{PATH_LOCAL_DATA}/data/train_data_juris_tcu_index.zip"
PATH_TRAIN_DATA = f"{PATH_LOCAL_DATA}/data/train_juris_tcu_index/train_data_juris_tcu_index.csv"

In [38]:
os.path.exists(PATH_TRAIN_DATA)

True

In [ ]:
%%time
if not os.path.exists(PATH_TRAIN_DATA):
  import zipfile
  !wget https://github.com/marcusborela/ind-ir/raw/main/data/train_juris_tcu_index/train_data_juris_tcu_index.csv -O {PATH_TRAIN_DATA}

  # Extrair o arquivo zip
  with zipfile.ZipFile(PATH_TRAIN_DATA_ZIP, 'r') as zip_ref:
      zip_ref.extractall(PATH_LOCAL_DATA)

  # Listar os arquivos extraídos
  arquivos_extraidos = zip_ref.namelist()
  # Exibir os arquivos extraídos
  for arquivo in arquivos_extraidos:
      print(arquivo)
  print("File loaded")
else:
  print("File already there!")

File already there!
CPU times: user 866 µs, sys: 0 ns, total: 866 µs
Wall time: 1.2 ms


In [39]:
df_data = pd.read_csv(PATH_TRAIN_DATA)

In [40]:
df_data.shape

(402738, 7)

Verificando correção do arquivo!

In [41]:
print(df_data.isnull().sum())

QUERY_ID      0
DOC_ID        0
RELEVANCE     0
SCORE         0
TYPE          0
DOC_TEXT      0
QUERY_TEXT    0
Length: 7, dtype: int64


In [42]:
df_data[['QUERY_TEXT','DOC_TEXT']].applymap(len).describe()

,QUERY_TEXT,DOC_TEXT
count,402738.0000000,402738.0000000
mean,322.8252313,830.6957451
std,165.8299958,398.1844365
min,41.0000000,86.0000000
25%,217.0000000,572.0000000
50%,294.0000000,759.0000000
75%,391.0000000,1020.0000000
max,4212.0000000,3739.0000000


Para cada positivo, tem 2 negativos

In [43]:
df_data['RELEVANCE'].describe()

count    402738.0000000
mean          0.1666667
std           0.3726785
min           0.0000000
25%           0.0000000
50%           0.0000000
75%           0.0000000
max           1.0000000
Name: RELEVANCE, Length: 8, dtype: float64

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
df_data[['QUERY_TEXT','DOC_TEXT']]

,QUERY_TEXT,DOC_TEXT
0,O dever de observância à hierarquia militar nã...,"O termo é ""Agente público"".\nAgente público te..."
1,O dever de observância à hierarquia militar nã...,"O termo é ""Reforma-prêmio"".\nReforma-prêmio te..."
2,O dever de observância à hierarquia militar nã...,"O termo é ""Exercício financeiro anterior"".\nEx..."
3,O dever de observância à hierarquia militar nã...,"O termo é ""CJF"".\nCJF é classificado como uma ..."
4,O dever de observância à hierarquia militar nã...,"O termo é ""Embratur"".\nEmbratur é classificado..."
...,...,...
402733,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Oitiva"".\nOitiva tem definição: ""Co..."
402734,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Derrogação"".\nDerrogação tem nota d..."
402735,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Presunção relativa"".\nPresunção rel..."
402736,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Contas extraordinárias"".\nContas ex..."


# Separating train and valid


In [46]:
hparam['percent_test_size'] = .005 #@param [.10, .15, .20] {type:'raw'}

In [47]:
X_train, X_valid, Y_train, Y_valid = train_test_split(df_data[['QUERY_TEXT','DOC_TEXT']].values,
                                                      df_data['RELEVANCE'].values,
                                                      test_size=hparam['percent_test_size'],
                                                      stratify=df_data['RELEVANCE'].values, random_state=num_semente)


In [48]:
X_train.shape

(400724, 2)

In [49]:
X_train.shape, type(X_valid), type(Y_valid), X_valid.shape, Y_valid.shape

((400724, 2), numpy.ndarray, numpy.ndarray, (2014, 2), (2014,))

In [50]:
print(np.unique(Y_train, return_counts=True), '\n', np.unique(Y_valid, return_counts=True))

(array([0, 1]), array([333937,  66787])) 
 (array([0, 1]), array([1678,  336]))


In [51]:
X_train[:1]

array([['Não é vedado o pagamento de serviços extraordinários a servidor ocupante de função comissionada.',
        'O termo é "Afastamento".\nAfastamento tem nota de escopo: "Usar como termo modificador subordinado a determinados assuntos. No caso de afastamento cautelar e temporário, previsto no art. 44 da Lei 8.443/92, usar Afastamento de responsável. Para situações genéricas dos afastamentos previstos na Lei 8.112/90,  usar Afastamento de pessoal. Usar Afastamento do país para a saída do Brasil que não seja na situação tratada nos arts. 95 e 96 da Lei 8.112/90, nesse caso usar Afastamento para estudo ou missão no exterior.".\nAfastamento tem tradução em espanhol: "Eliminación".\nAfastamento tem tradução em inglês: "Removal".']],
      dtype=object)

In [52]:
Y_train[:2]

array([0, 0])

# Tokenizer and Dataset

#### Carga

In [53]:
hparam['model_name'] = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'

In [54]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [57]:
# se local
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + hparam['model_name']
assert os.path.exists(nome_caminho_modelo), f"Path para {hparam['model_name']} não existe!"


In [58]:
# tokenizer = AutoTokenizer.from_pretrained(hparam['model_name'])
# se local
tokenizer = AutoTokenizer.from_pretrained(nome_caminho_modelo)

loading file sentencepiece.bpe.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


#### Experimentações e computando limites

In [59]:
tokenizer.pad_token_id,tokenizer.cls_token_id,tokenizer.sep_token_id

(1, 0, 2)

In [60]:
x='x, it is just a test!'
y='y, it is just a continuation!'

In [61]:
tokenizer(x,y)['input_ids']

[0,
 1022,
 4,
 442,
 83,
 1660,
 10,
 3034,
 38,
 2,
 2,
 113,
 4,
 442,
 83,
 1660,
 10,
 9454,
 1363,
 38,
 2]

In [62]:
tokenizer.encode_plus(x,y)['input_ids']

[0,
 1022,
 4,
 442,
 83,
 1660,
 10,
 3034,
 38,
 2,
 2,
 113,
 4,
 442,
 83,
 1660,
 10,
 9454,
 1363,
 38,
 2]

### Definindo parâmetros e informações para o rastro (hparam) - sobre dados

In [63]:
# visto que "max_position_embeddings": 512 em https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/blob/main/config.json

hparam['vocab_size'] = tokenizer.vocab_size
hparam['batch_size'] = 32
hparam['num_epochs'] = 10  #@param {type:"slider", min:1, max:100, step:1}
hparam['max_seq_length'] = 512  #tokenizer.max_model_input_sizes
hparam['num_sentenca_train'] = X_train.shape[0] # ou 37400
hparam['num_sentenca_valid'] = X_valid.shape[0] #


Definindo o prefixo de nome do arquivo

In [64]:
PATH_LOCAL_DATA

'../..'

In [65]:
PATH_TRAIN_LOCAL = f"{PATH_LOCAL_DATA}/model/train/minilm"

In [66]:
if not os.path.exists(PATH_TRAIN_LOCAL):
  os.makedirs(PATH_TRAIN_LOCAL)
  print(f'{PATH_TRAIN_LOCAL} pasta criada!')
else:
  print(f'{PATH_TRAIN_LOCAL} pasta já existia!')


../../model/train/minilm pasta criada!


In [ ]:
os.path.exists(PATH_TRAIN_LOCAL)

True

## Criando código para o Dataset

In [69]:
class MyDataset():
    """
      Classe para representar um dataset de texto e classes.
    """
    def __init__(self, texts: np.ndarray, classes:list[int], tokenizer, max_seq_length: int):
      """
      Inicializa um novo objeto MyDataset.

      Args:
          texts (np.ndarray): um array com as strings de texto. Cada linha deve ter 2 strings.
          classes (np.ndarray): um array com as classes de cada texto.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
          max_seq_length (int): o tamanho máximo da sequência a ser considerado.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      assert isinstance(texts, np.ndarray), f"Parâmetro texts deve ser do tipo np.ndarray e não {type(texts)}"
      assert texts.shape[1] == 2, "Array must have 2 columns"
      for row in texts:
          assert isinstance(row, np.ndarray) and row.shape == (2,), f"Each row in texts must have 2 elements"
          assert isinstance(row[0], str) and isinstance(row[1], str), f"Each element in texts.row must be a string e não {type(row[0])}"
      assert isinstance(classes,np.ndarray), f'classes deve ser do tipo np.ndarray e não {type(classes)}'
      assert isinstance(classes[0],np.int64), f'classes[0] deve ser do tipo numpy.int64 e não {type(classes[0])} '
      assert isinstance(max_seq_length, int), f'max_seq_length deve ser do tipo int e não {type(max_seq_length)}'
      assert max_seq_length > 100, f'max_seq_length deve ser maior do que 100'

      self.texts = texts
      self.classes = classes
      self.tokenizer = tokenizer
      self.max_seq_length = max_seq_length

      # Salvar os dados dos tensores
      x_data_input_ids = []
      x_data_token_type_ids = []
      x_data_attention_masks = []
      for text_pair in tqdm(texts, desc='encoding text pair'):
          encoding = tokenizer.encode_plus(
              text_pair[0],
              text_pair[1],
              add_special_tokens=True,
              max_length=self.max_seq_length,
              padding='max_length',
              return_tensors = 'pt',
              truncation=True,
              return_attention_mask=True,
              return_token_type_ids=True
          )
          x_data_input_ids.append(encoding['input_ids'].long())
          x_data_token_type_ids.append(encoding['token_type_ids'].long())
          x_data_attention_masks.append(encoding['attention_mask'].long())
      print(F'\tVou converter lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      # squeeze: vai transformar um tensor de shape [2, 1, 322] em um tensor de shape [2, 322].

      self.x_tensor_input_ids = torch.stack(x_data_input_ids).squeeze(1)
      self.x_tensor_attention_masks = torch.stack(x_data_attention_masks).squeeze(1)
      self.x_tensor_token_type_ids = torch.stack(x_data_token_type_ids).squeeze(1)

    def __len__(self):
        """
          Retorna o tamanho do dataset (= tamanho do array texts)
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
          Retorna um dicionário com os dados do texto e sua classe correspondente, em um formato que pode
          ser usado pelo dataloader do PyTorch para alimentar um modelo de aprendizado de máquina.
        """
        return {
            'input_ids': self.x_tensor_input_ids[idx],
            'attention_mask': self.x_tensor_attention_masks[idx],
            'token_type_ids': self.x_tensor_token_type_ids[idx],
            # 'labels': int(self.classes[idx])
            # 'labels': torch.tensor(self.classes[idx], dtype=torch.float)
            # 'labels' : torch.tensor(-1. if self.classes[idx] == 0 else 1., dtype=torch.float)
            'labels' : torch.tensor(self.classes[idx], dtype=torch.float)
        }

#### Testando o MyDataset e o Dataloader

In [70]:
# Cria dados fictícios
texts = np.array([['This is the first text', 'This is the second text'],
                  ['This is text 2.1', 'This is text 2.2'],
                  ['This is text 3.1', 'This is text 3.2'],
                  ['This is text 4.1', 'This is text 4.2'],
                  ['This is text 5.1', 'This is text 5.2'],
                  ['This is text 6.1', 'This is text 6.2'],
                  ['This is text 7.1', 'This is text 7.2']])
classes = np.array([1, 0, 1, 0, 1, 0, 1])


In [71]:
# Cria um objeto da classe MyDataset
dummy_dataset = MyDataset(texts=texts, classes=classes, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])

encoding text pair: 100%|██████████| 7/7 [00:00<00:00, 39.30it/s]


	Vou converter lista para tensor;  Momento: [2023-Jun-27 00:42:39]


In [72]:
# Testa o método __len__()
assert len(dummy_dataset) == 7

# Testa o método __getitem__()
sample = dummy_dataset[0]
assert set(sample.keys()) == {'input_ids', 'attention_mask', 'token_type_ids', 'labels'} #
assert isinstance(sample['input_ids'], torch.Tensor)
assert sample['input_ids'].shape[0] == hparam['max_seq_length']
assert isinstance(sample['attention_mask'], torch.Tensor)
assert sample['attention_mask'].shape[0] == hparam['max_seq_length']
# assert isinstance(sample['labels'], int)
assert isinstance(sample['labels'], torch.Tensor)
# print(sample['labels'], sample['labels'].shape)


In [73]:
print(sample)

{'input_ids': tensor([    0,  3293,    83,    70,  5117,  7986,     2,     2,  3293,    83,
           70, 17932,  7986,     2,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1, 

In [74]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers=hparam['num_workers_dataloader'] )

In [77]:
from transformers import BatchEncoding

In [78]:
first_batch = next(iter(dummy_loader))

In [79]:
BatchEncoding(first_batch).to(hparam['device'])

{'input_ids': tensor([[   0, 3293,   83,  ...,    1,    1,    1],
        [   0, 3293,   83,  ...,    1,    1,    1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([1.0000e+00, 0.0000e+00], device='cuda:0')}

In [80]:
first_batch

{'input_ids': tensor([[   0, 3293,   83,  ...,    1,    1,    1],
         [   0, 3293,   83,  ...,    1,    1,    1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'labels': tensor([1.0000e+00, 0.0000e+00])}

Confirmando efeito do parâmetro shuffle em DataLoader

In [81]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers=hparam['num_workers_dataloader'])
for ndx, epoch in tqdm(enumerate(range(hparam['num_epochs'])), desc='Epochs'):
  for batch in dummy_loader:
    print(batch['labels'])
  print(f"Fim época {ndx}")


Epochs: 10it [00:00, 1035.73it/s]

tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00])
Fim época 0
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00])
Fim época 1
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00])
Fim época 2
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00])
Fim época 3
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00])
Fim época 4
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00])
Fim época 5
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00])
Fim época 6
tensor([1.0000e+00, 0.0000e+00])
tensor([1.0000e+00, 0.0000e+00])
tensor([1.

In [ ]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=True, num_workers=hparam['num_workers_dataloader'])
for ndx, epoch in tqdm(enumerate(range(hparam['num_epochs'])), desc='Epochs'):
  for batch in dummy_loader:
    print(batch['labels'])
  print(f"Fim época {ndx}")

Epochs: 10it [00:00, 195.58it/s]

tensor([-1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 0
tensor([-1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00])
Fim época 1
tensor([-1.0000e+00, -1.0000e+00])
tensor([-1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([1.0000e+00])
Fim época 2
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 3
tensor([1.0000e+00, -1.0000e+00])
tensor([-1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 4
tensor([1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, -1.0000e+00])
tensor([-1.0000e+00])
Fim época 5
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00])
Fim época 6
tensor([-1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, 

In [82]:
del dummy_loader, dummy_dataset

# Treinamento do ranqueador

Fonte: https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing

In this notebook, we will finetuned and evaluate a BERT-based model on a document classification task.

Our dataset will be a smaller version of the IMDB Sentiment Analysis dataset.

To finetune and evaluate faster, we will be using [miniLM](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased), which is small BERT-like model distilled from a larger one.

## Carregando o Dataset e os Dataloaders


Independente da configuração do leiaute de treinamento, a validação é no leiaute qp

In [83]:
datasets_carregados_previamente = False #@param {type:"boolean"}

print(datasets_carregados_previamente)

False


In [84]:
import io

In [85]:
%%time
if datasets_carregados_previamente:
  with open(PATH_TRAIN_LOCAL+'/dataset_valid_len_'+str(hparam['num_sentenca_valid'])+'.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_valid = torch.load(buffer)
  with open(PATH_TRAIN_LOCAL+'/dataset_train'+'_len_'+str(hparam['num_sentenca_train'])+'.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_train = torch.load(buffer)
  print('Datasets carregados')

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs


In [87]:
%%time
if not datasets_carregados_previamente:
  print("carregando valid_dataset")
  dataset_valid = MyDataset(texts=X_valid, classes=Y_valid, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(dataset_valid, PATH_TRAIN_LOCAL+'/dataset_valid_len_'+str(hparam['num_sentenca_valid'])+'.pt')

carregando valid_dataset


encoding text pair: 100%|██████████| 2014/2014 [00:01<00:00, 1283.63it/s]


	Vou converter lista para tensor;  Momento: [2023-Jun-27 00:44:00]
CPU times: user 1.62 s, sys: 37.9 ms, total: 1.65 s
Wall time: 1.65 s


In [88]:
%%time
if not datasets_carregados_previamente:
  print("carregando train_dataset")
  dataset_train = MyDataset(texts=X_train, classes=Y_train, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(dataset_train, PATH_TRAIN_LOCAL+'/dataset_train'+'_len_'+str(hparam['num_sentenca_train'])+'.pt')

carregando train_dataset


encoding text pair: 100%|██████████| 400724/400724 [06:37<00:00, 1007.60it/s]


	Vou converter lista para tensor;  Momento: [2023-Jun-27 00:51:03]
CPU times: user 6min 40s, sys: 4.52 s, total: 6min 45s
Wall time: 6min 44s


## Finetuning

### Funções auxiliares de treinamento

In [89]:
gera_tag_rastro_experiencia_treino(parm_aula='indir-minilm', hparam=hparam)


'indir-minilm|max_seq_length 512|batch_size 32'

In [146]:
def treina_modelo (model, hparam, parm_dataloader_train, parm_dataloader_valid,
                   parm_intervalo_print:int=1,
                   parm_se_salva_modelo:bool=True, parm_se_gera_rastro:bool=True):
    global PATH_TRAIN_LOCAL

    if parm_se_gera_rastro:
        rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= gera_tag_rastro_experiencia_treino(parm_aula='indir-minilm', hparam=hparam) )
    history = []
    n_examples = 0
    time_inicio_treino = time.time()



    try:
        best_metrica_valid = 0 # float('-inf')
        best_epoch = 0


        train_losses = []
        correct = 0
        train_samples = 0
        model.train()
        se_para_execucao = False
        ultimo_step_treinado = 0
        for epoch in tqdm(range(hparam['num_epochs']), desc='Epochs'):

            if n_examples >= hparam['max_examples'] or se_para_execucao:
                break


            for cnt_step, batch in enumerate(tqdm(parm_dataloader_train, mininterval=0.5, desc='Train', disable=False)):

                if n_examples >= hparam['max_examples'] or se_para_execucao:
                    break
                ultimo_step_treinado += 1
                hparam['optimizer'].zero_grad()
                # ipdb.set_trace(context=3)
                outputs = model(input_ids=batch['input_ids'].to(hparam['device']),
                                attention_mask=batch['attention_mask'].to(hparam['device']),
                                token_type_ids=batch['token_type_ids'].to(hparam['device']),
                                labels=batch['labels'].to(hparam['device']))
                loss = outputs.loss
                loss.backward()
                hparam['optimizer'].step()
                hparam['scheduler'].step()
                n_examples += len(batch['input_ids'])  # Increment of batch size
                train_losses.append(loss.cpu().item())
                train_samples += batch['labels'].size(0)

                # se saída for 0 e 1
                preds = (outputs.logits >= 0.5).float().squeeze().to(hparam['device'])
                correct += (preds == batch['labels'].to(hparam['device'])).sum().cpu().item()

                # se saída for -1 e 1
                # preds = torch.sign(outputs.logits).squeeze()
                # correct += (preds == labels).sum().item()



                # ipdb.set_trace(context=2)
                if ultimo_step_treinado % hparam['eval_every_steps']==0:
                    if isinstance(parm_dataloader_train,torch.utils.data.dataloader.DataLoader):
                        accuracy_train = correct / train_samples
                    elif isinstance(parm_dataloader_train,list):  # usado para overfit em um batch
                        accuracy_train = correct / train_samples
                        assert len(parm_dataloader_train) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"
                    else:
                        raise Exception(f'parm_dataloader_train has type unexpected {type(parm_dataloader_train)} ')
                    loss_train_mean = mean(train_losses)
                    train_losses = []
                    correct = 0
                    train_samples = 0
                    accuracy_valid, mean_loss_valid = evaluate(model=model, dataloader=parm_dataloader_valid, set_name='Valid')
                    model.train()
                    # ipdb.set_trace(context=2)
                    print(f'Epoch: {epoch + 1} Step: {ultimo_step_treinado} Training loss: {loss_train_mean:0.2f} accuracy: {accuracy_train:0.3f} \nEm validação: loss: {mean_loss_valid:0.3f}; accuracy: {accuracy_valid:0.3f}')
                    metrica_rastro = {"train/loss": loss_train_mean,
                                      "train/accuracy": accuracy_train,
                                      "train/steps": ultimo_step_treinado,
                                      "train/n_examples": n_examples,
                                      "train/learning_rate": hparam["optimizer"].param_groups[0]["lr"],
                                      "valid/loss": mean_loss_valid,
                                      "valid/accuracy": accuracy_valid}
                    history.append(metrica_rastro)
                    if parm_se_gera_rastro:
                        rastro_neptune.salva_metrica(metrica_rastro)

                    if parm_intervalo_print > 0:
                        if (ultimo_step_treinado)%(parm_intervalo_print*hparam['eval_every_steps']) == 0:
                            print(f'Step: {ultimo_step_treinado} Amostras:{n_examples:d}  {100*n_examples/hparam["max_examples"]:.3f}%  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
                            print(f'lr: {hparam["optimizer"].param_groups[0]["lr"]:.5e} Train loss: {loss_train_mean:.4f}  accuracy: {accuracy_train:.4f}')
                            print(f'Valid loss: {mean_loss_valid:.4f} accuracy: {accuracy_valid:.4f}  ')

                    if accuracy_valid > best_metrica_valid:
                        best_model_dict = model.state_dict()
                        best_metrica_valid = accuracy_valid
                        best_step = ultimo_step_treinado
                        if parm_se_salva_modelo:
                          path_modelo = f'{PATH_TRAIN_LOCAL}/model_epoch_{best_step}_treino_{time.strftime("%Y-%b-%d %H:%M:%S")}.pt'
                          # torch.save(model, path_modelo)
                          model.save_pretrained(path_modelo)

                        # print(f'encontrado best model Epoch: {epoch + 1} ')
                    elif hparam['early_stop'] < ( ultimo_step_treinado + 1 - best_step):
                        print(f"Parando por critério de early_stop na época {epoch + 1} step {ultimo_step_treinado} sendo best_step {best_step} e ealy_stop {hparam['early_stop']}")
                        se_para_execucao = True

        # fim do treino
        if isinstance(parm_dataloader_train,torch.utils.data.dataloader.DataLoader):
            accuracy_train = correct / len(parm_dataloader_train.dataset)
        elif isinstance(parm_dataloader_train,list):  # usado para overfit em um batch
            accuracy_train = correct / len(parm_dataloader_train[0]['labels'])
            assert len(parm_dataloader_train) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"

        #loss_train_mean = mean(train_losses)
        accuracy_valid, mean_loss_valid = evaluate(model=model, dataloader=parm_dataloader_valid, set_name='Valid')

        print(f'Epoch: {epoch + 1} Step: {ultimo_step_treinado} Training loss: {loss_train_mean:0.2f} accuracy: {accuracy_train:0.3f} \nEm validação: loss: {mean_loss_valid:0.3f}; accuracy: {accuracy_valid:0.3f}')
        metrica_rastro = {"train/loss": loss_train_mean,
                          "train/accuracy": accuracy_train,
                          "train/steps": ultimo_step_treinado,
                          "train/n_examples": n_examples,
                          "train/learning_rate": hparam["optimizer"].param_groups[0]["lr"],
                          "valid/loss": mean_loss_valid,
                          "valid/accuracy": accuracy_valid}
        history.append(metrica_rastro)
        if parm_se_gera_rastro:
            rastro_neptune.salva_metrica(metrica_rastro)

        print(f'** END ** Step: {ultimo_step_treinado} Amostras:{n_examples:d}  {100*n_examples/hparam["max_examples"]:.3f}%  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
        print(f'lr: {hparam["optimizer"].param_groups[0]["lr"]:.5e} Train loss: {loss_train_mean:.4f}  accuracy: {accuracy_train:.4f}')
        print(f'Valid loss: {mean_loss_valid:.4f} accuracy: {accuracy_valid:.4f}  ')


        if parm_se_salva_modelo:
            path_modelo = f'{PATH_TRAIN_LOCAL}/model_fim_treino_{time.strftime("%Y-%b-%d %H:%M:%S")}.pt'
            # torch.save(model, path_modelo)
            model.save_pretrained(path_modelo)
            print(f"Modelo final  (step {ultimo_step_treinado}) salvo em {path_modelo}")

            if best_model_dict is not None:
                model.load_state_dict(best_model_dict)
                model.to(hparam['device'])
                path_modelo = f'{PATH_TRAIN_LOCAL}/model_fim_treino_{time.strftime("%Y-%b-%d %H:%M:%S")}.pt'
                # torch.save(model, path_modelo)
                model.save_pretrained(path_modelo)
                print(f"Modelo com melhor resultado em validação (step {best_step}) salvo após treino em {path_modelo}")

        # calculando tempo gasto e médio por
        tempo_treino = time.time() - time_inicio_treino
        if parm_se_gera_rastro:
            rastro_neptune.run_neptune["context/tempo_treino"] = tempo_treino
            rastro_neptune.run_neptune["context/best_epoch"] = best_epoch
            rastro_neptune.run_neptune["context/tempo_treino_epoc_mean"] = tempo_treino/hparam['num_epochs']
            # rastro_neptune.gera_grafico_modelo(parm_dataloader_train, model)
    finally: # para não deixar em aberto a execução no Neptune
        if parm_se_gera_rastro:
            rastro_neptune.stop()
    return history

In [124]:
def count_parameters(model):
    """
    Conta o número de parâmetros treináveis em um modelo PyTorch.

    Args:
        model (torch.nn.Module): O modelo PyTorch a ser avaliado.

    Returns:
        int: O número de parâmetros treináveis do modelo.

    """
    # Retorna a soma do número de elementos em cada tensor de parâmetro que requer gradiente.
    # A propriedade "requires_grad" é definida como True para todos os tensores de parâmetro que
    # precisam ser treinados, enquanto que para aqueles que não precisam, ela é definida como False.
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [125]:
from statistics import mean, stdev

In [126]:
# We first define the evaluation function to measure accuracy and loss
def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for ndx, batch in tqdm(enumerate(dataloader), mininterval=0.5, desc=set_name, disable=False, total=len(dataloader)):
            input_ids = batch['input_ids'].to(hparam['device'])
            token_type_ids = batch['token_type_ids'].to(hparam['device'])
            attention_mask = batch['attention_mask'].to(hparam['device'])
            labels = batch['labels'].to(hparam['device'])
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())

            # se saída for 0 e 1
            preds = (outputs.logits >= 0.5).float().squeeze().to(hparam['device'])
            correct += (preds == labels).sum().item()

            # se saída for -1 e 1
            # preds = torch.sign(outputs.logits).squeeze()
            # correct += (preds == labels).sum().item()


            # if ndx < 5:
            #   print(f'Em evaluete. Batch {ndx}')
            #   print(f'loss_val: {loss_val} ')
            #   print(f'preds: {preds} \nlabels: {labels} ')
            #   print(f'correct: {correct}')

    # ipdb.set_trace(context=3)
    if isinstance(dataloader,torch.utils.data.dataloader.DataLoader):
        accuracy = correct / len(dataloader.dataset)
    elif isinstance(dataloader,list):  # usado para overfit em um batch
        accuracy = correct / len(dataloader[0]['labels'])
        assert len(dataloader) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"
    mean_loss = mean(losses)
    return accuracy, mean_loss

In [127]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

In [128]:
def inicializa_modelo():
  global model, hparam, dataloader_train, nome_caminho_modelo

  model = AutoModelForSequenceClassification.from_pretrained(nome_caminho_modelo).to(hparam['device'])
  hparam['learning_rate']= 1e-6 # 3e-5 # 1e-3
  hparam['num_params'] = count_parameters(model)
  print(f"Number of model parameters: {hparam['num_params']}")
  hparam['amsgrad']= False
  hparam['num_epochs'] = 4
  hparam['weight_decay'] = 1e-4
  hparam['early_stop'] = 8000
  hparam['batch_size'] = 32
  hparam['num_step_epoch'] = len(dataloader_train)
  hparam['num_training_steps'] = hparam['num_epochs'] * hparam['num_step_epoch']
  hparam['max_examples'] = hparam['num_training_steps'] * hparam['batch_size']
  hparam['eval_every_steps'] = 400
  hparam['num_warmup_steps'] = 400 # int(hparam['num_training_steps']  * 0.05)
  hparam['optimizer'] = torch.optim.Adam(model.parameters(), lr=hparam['learning_rate'], weight_decay= hparam['weight_decay'], amsgrad=hparam['amsgrad'])
  hparam['scheduler'] = get_linear_schedule_with_warmup(hparam['optimizer'], hparam['num_warmup_steps'], hparam['num_training_steps'])
  # hparam['step_eval'] = 1000


In [99]:
dataloader_train = DataLoader(dataset_train, batch_size=hparam['batch_size'], shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=hparam['batch_size'], shuffle=False)

In [103]:
import gc

In [111]:
gc.collect()

5174

In [105]:
inicializa_modelo()

loading configuration file /home/borela/fontes/relevar-busca/modelo/unicamp-dl/mMiniLM-L6-v2-pt-v2/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "/home/borela/fontes/relevar-busca/modelo/unicamp-dl/mMiniLM-L6-v2-pt-v2",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sbert_ce_default_activation_function": "torch.nn.modules.linear.Identity",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 

Number of model parameters: 106994305


Depois tentar definir:hparam[ hparam['dim_feedforward']=256

In [ ]:
raise Exception('parar aqui')

Exception: ignored

### Evaluating on the valid dataset before training

In [112]:
%%time
accuracy, mean_loss = evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')
print(f'Em validação: loss: {mean_loss:0.3f}; accuracy: {accuracy:0.3f}')

Valid: 100%|██████████| 63/63 [00:09<00:00,  6.63it/s]

Em validação: loss: 58.289; accuracy: 0.823
CPU times: user 4.9 s, sys: 320 ms, total: 5.22 s
Wall time: 9.5 s


### Testando em poucos dados: um batch apenas. Esperado overfit loss próxima de zero.

Por algum motivo a investigar quando passo dataset como lista não está gerando aprendizado. Mas quando passo normal, o modelo está aprendendo.




In [113]:
hparam['num_epochs'] = 5
hparam['early_stop'] = 3
hparam['eval_every_steps'] = 1

In [ ]:
history =  treina_modelo (model, hparam,  [next(iter(dataloader_train))], [next(iter(dataloader_valid))], parm_se_salva_modelo=True, parm_se_gera_rastro=True)

Treinando em todos os dados

ultimo_step_treinado , correct, cnt_step, n_examples, train_losses
accuracy_train, accuracy_valid, mean_loss_valid



In [136]:
torch.cuda.empty_cache()

In [137]:
gc.collect()

3817

In [138]:
inicializa_modelo()

loading configuration file /home/borela/fontes/relevar-busca/modelo/unicamp-dl/mMiniLM-L6-v2-pt-v2/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "/home/borela/fontes/relevar-busca/modelo/unicamp-dl/mMiniLM-L6-v2-pt-v2",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sbert_ce_default_activation_function": "torch.nn.modules.linear.Identity",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 

Number of model parameters: 106994305


In [139]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 41.29
 used 24.82
 free 13.67
 cached 27.89
 buffers 0.97
/nGPU
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Jun 27 01:26:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.03   Driver Version: 525.116.03   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+==================

### Treino do conjunto completo

In [140]:
hparam

{'num_workers_dataloader': 0,
 'device': device(type='cuda', index=0),
 'percent_test_size': 0.005,
 'model_name': 'unicamp-dl/mMiniLM-L6-v2-pt-v2',
 'vocab_size': 250002,
 'batch_size': 32,
 'num_epochs': 4,
 'max_seq_length': 512,
 'num_sentenca_train': 400724,
 'num_sentenca_valid': 2014,
 'learning_rate': 1e-06,
 'num_params': 106994305,
 'amsgrad': False,
 'weight_decay': 0.0001,
 'early_stop': 8000,
 'num_step_epoch': 12523,
 'num_training_steps': 50092,
 'max_examples': 1602944,
 'eval_every_steps': 400,
 'num_warmup_steps': 400,
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     initial_lr: 1e-06
     lr: 0.0
     maximize: False
     weight_decay: 0.0001
 ),
 'scheduler': <torch.optim.lr_scheduler.LambdaLR at 0x7f840460ba60>}

In [141]:
history = treina_modelo (model, hparam,  dataloader_train, dataloader_valid, parm_intervalo_print=1)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epochs:   0%|          | 0/4 [00:00<?, ?it/s]







Valid: 100%|██████████| 63/63 [00:03<00:00, 16.63it/s]
Configuration saved in ../../model/train/minilm/model_epoch_400_treino_2023-Jun-27 01:29:13.pt/config.json


Epoch: 1 Step: 400 Training loss: 27.61 accuracy: 0.727 
Em validação: loss: 6.575; accuracy: 0.610
Step: 400 Amostras:12800  0.799%  Momento: [2023-Jun-27 01:29:13]
lr: 1.00000e-06 Train loss: 27.6101  accuracy: 0.7274
Valid loss: 6.5750 accuracy: 0.6097  


Model weights saved in ../../model/train/minilm/model_epoch_400_treino_2023-Jun-27 01:29:13.pt/pytorch_model.bin








Valid: 100%|██████████| 63/63 [00:03<00:00, 16.54it/s]
Configuration saved in ../../model/train/minilm/model_epoch_800_treino_2023-Jun-27 01:31:22.pt/config.json


Epoch: 1 Step: 800 Training loss: 2.80 accuracy: 0.613 
Em validação: loss: 1.697; accuracy: 0.622
Step: 800 Amostras:25600  1.597%  Momento: [2023-Jun-27 01:31:22]
lr: 9.91950e-07 Train loss: 2.7960  accuracy: 0.6128
Valid loss: 1.6968 accuracy: 0.6216  


...

Epoch: 4 Step: 50092 Training loss: 0.04 accuracy: 0.007 
Em validação: loss: 0.046; accuracy: 0.943
** END ** Step: 50092 Amostras:1602896  99.997%  Momento: [2023-Jun-27 06:00:04]
lr: 0.00000e+00 Train loss: 0.0439  accuracy: 0.0069
Valid loss: 0.0462 accuracy: 0.9429  


Model weights saved in ../../model/train/minilm/model_fim_treino_2023-Jun-27 06:00:04.pt/pytorch_model.bin
Configuration saved in ../../model/train/minilm/model_fim_treino_2023-Jun-27 06:00:04.pt/config.json


Modelo final  (step 50092) salvo em ../../model/train/minilm/model_fim_treino_2023-Jun-27 06:00:04.pt


Model weights saved in ../../model/train/minilm/model_fim_treino_2023-Jun-27 06:00:04.pt/pytorch_model.bin


Modelo com melhor resultado em validação (step 49200) salvo após treino em ../../model/train/minilm/model_fim_treino_2023-Jun-27 06:00:04.pt
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 32 operations to synchronize with Neptune. Do not kill this process.
All 32 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-104


In [142]:
print(history)

[{'train/loss': 27.61007399559021, 'train/accuracy': 0.727421875, 'train/steps': 400, 'train/n_examples': 12800, 'train/learning_rate': 1e-06, 'valid/loss': 6.575049271659245, 'valid/accuracy': 0.6097318768619663}, {'train/loss': 2.796027316749096, 'train/accuracy': 0.6128125, 'train/steps': 800, 'train/n_examples': 25600, 'train/learning_rate': 9.919504145536505e-07, 'valid/loss': 1.6967824848871382, 'valid/accuracy': 0.6216484607745779}, {'train/loss': 0.9683212437480688, 'train/accuracy': 0.65265625, 'train/steps': 1200, 'train/n_examples': 38400, 'train/learning_rate': 9.839008291073008e-07, 'valid/loss': 0.8495350532115452, 'valid/accuracy': 0.6261171797418074}, {'train/loss': 0.5432193081453442, 'train/accuracy': 0.691015625, 'train/steps': 1600, 'train/n_examples': 51200, 'train/learning_rate': 9.758512436609514e-07, 'valid/loss': 0.5295586685339609, 'valid/accuracy': 0.6658391261171798}, {'train/loss': 0.37736979246139524, 'train/accuracy': 0.7171875, 'train/steps': 2000, 'trai

In [143]:
gc.collect()

0

In [144]:
torch.cuda.empty_cache()

In [145]:
history2 = treina_modelo (model, hparam,  dataloader_train, dataloader_valid, parm_intervalo_print=1)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epochs:   0%|          | 0/4 [00:00<?, ?it/s]







Valid: 100%|██████████| 63/63 [00:03<00:00, 16.53it/s]
Configuration saved in ../../model/train/minilm/model_epoch_400_treino_2023-Jun-27 08:25:37.pt/config.json


Epoch: 1 Step: 400 Training loss: 0.04 accuracy: 0.947 
Em validação: loss: 0.046; accuracy: 0.943
Step: 400 Amostras:12800  0.799%  Momento: [2023-Jun-27 08:25:37]
lr: 0.00000e+00 Train loss: 0.0443  accuracy: 0.9467
Valid loss: 0.0462 accuracy: 0.9429  


Model weights saved in ../../model/train/minilm/model_epoch_400_treino_2023-Jun-27 08:25:37.pt/pytorch_model.bin








Valid: 100%|██████████| 63/63 [00:03<00:00, 16.63it/s]


Epoch: 1 Step: 800 Training loss: 0.05 accuracy: 0.941 
Em validação: loss: 0.046; accuracy: 0.943
Step: 800 Amostras:25600  1.597%  Momento: [2023-Jun-27 08:27:45]
lr: 0.00000e+00 Train loss: 0.0483  accuracy: 0.9407
Valid loss: 0.0462 accuracy: 0.9429  


...

Epoch: 1 Step: 8000 Training loss: 0.05 accuracy: 0.946 
Em validação: loss: 0.046; accuracy: 0.943
Step: 8000 Amostras:256000  15.971%  Momento: [2023-Jun-27 09:06:51]
lr: 0.00000e+00 Train loss: 0.0461  accuracy: 0.9458
Valid loss: 0.0462 accuracy: 0.9429  










Epochs:  25%|██▌       | 1/4 [45:33<2:16:40, 2733.43s/it]

Epoch: 1 Step: 8400 Training loss: 0.05 accuracy: 0.947 
Em validação: loss: 0.046; accuracy: 0.943
Step: 8400 Amostras:268800  16.769%  Momento: [2023-Jun-27 09:09:02]
lr: 0.00000e+00 Train loss: 0.0450  accuracy: 0.9466
Valid loss: 0.0462 accuracy: 0.9429  
Parando por critério de early_stop na época 1 step 8400 sendo best_step 400 e ealy_stop 8000
Shutting down background jobs, please wait a moment...


Done!
Waiting for the remaining 56 operations to synchronize with Neptune. Do not kill this process.
All 56 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-105


StatisticsError: mean requires at least one data point

In [147]:
history3 = treina_modelo (model, hparam,  dataloader_train, dataloader_valid, parm_intervalo_print=1)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epochs:   0%|          | 0/4 [00:00<?, ?it/s]







Valid: 100%|██████████| 63/63 [00:03<00:00, 16.63it/s]
Configuration saved in ../../model/train/minilm/model_epoch_400_treino_2023-Jun-27 10:03:39.pt/config.json


Epoch: 1 Step: 400 Training loss: 0.05 accuracy: 0.944 
Em validação: loss: 0.046; accuracy: 0.943
Step: 400 Amostras:12800  0.799%  Momento: [2023-Jun-27 10:03:39]
lr: 0.00000e+00 Train loss: 0.0473  accuracy: 0.9443
Valid loss: 0.0462 accuracy: 0.9429  


Model weights saved in ../../model/train/minilm/model_epoch_400_treino_2023-Jun-27 10:03:39.pt/pytorch_model.bin








Valid: 100%|██████████| 63/63 [00:03<00:00, 16.62it/s]


Epoch: 1 Step: 800 Training loss: 0.05 accuracy: 0.944 
Em validação: loss: 0.046; accuracy: 0.943
Step: 800 Amostras:25600  1.597%  Momento: [2023-Jun-27 10:05:48]
lr: 0.00000e+00 Train loss: 0.0470  accuracy: 0.9439
Valid loss: 0.0462 accuracy: 0.9429  


...

Epoch: 2 Step: 8400 Training loss: 0.04 accuracy: 0.000 
Em validação: loss: 0.046; accuracy: 0.943
** END ** Step: 8400 Amostras:268800  16.769%  Momento: [2023-Jun-27 10:47:17]
lr: 0.00000e+00 Train loss: 0.0448  accuracy: 0.0000
Valid loss: 0.0462 accuracy: 0.9429  


Model weights saved in ../../model/train/minilm/model_fim_treino_2023-Jun-27 10:47:17.pt/pytorch_model.bin
Configuration saved in ../../model/train/minilm/model_fim_treino_2023-Jun-27 10:47:18.pt/config.json


Modelo final  (step 8400) salvo em ../../model/train/minilm/model_fim_treino_2023-Jun-27 10:47:17.pt


Model weights saved in ../../model/train/minilm/model_fim_treino_2023-Jun-27 10:47:18.pt/pytorch_model.bin


Modelo com melhor resultado em validação (step 400) salvo após treino em ../../model/train/minilm/model_fim_treino_2023-Jun-27 10:47:18.pt
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 32 operations to synchronize with Neptune. Do not kill this process.
All 32 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-106


In [148]:
%%time
accuracy, mean_loss = evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')
print(f'Em validação: loss: {mean_loss:0.3f}; accuracy: {accuracy:0.3f}')

Valid: 100%|██████████| 63/63 [00:03<00:00, 16.88it/s]

Em validação: loss: 0.046; accuracy: 0.943
CPU times: user 3.72 s, sys: 20.2 ms, total: 3.74 s
Wall time: 3.74 s
